This is originally a collaborative project with Isky on an implementation of the Hartree-Fock method, a way of approximating the wave function and thus energy of bonds in a quantum molecular system. However, despite a week of research, the complexity of the problem forces us to cease the plan, and instead resort to a much simpler task of plotting the probability density of hydrogen-like orbitals.

# Plotting the Atomic Orbital - Johnny Tse

This is a project which draws out the hull of each atomic orbital of a single electron in a hydrogen atom. This is done by obtaining the wave function of the electron, finding the (complex) probability amplitude at points around the atom, then to square the amplitude to derive the probability density. A contour of the proabability density is then draw to illustrate the region where the electron is most likely to be found.

It is inevitable that this would be an easy process. It is a difficult task to derive the exhaustive list of the orbitals' wave function which can obey the schrodinger equation in this situation. However, once derived, one can easily plot out the orbitals, and there is no easy way of finding the wave functions by simulation.

It did, however, allowed for deeper understanding of what the wave function is. Each electorn around a nucleus can be characterized by four variables, the principal quantum number (n), the azimuthal quantum number (l), the magnetic quantum number (m) and the spin quantum number (s). The principal quantum number is the average distance of the electron from nucleus/energy level, the azimuthal quantum number shows the orbital's shape/angular momentum, ,the magnetic quantum number is the orbital's orientation/angular momentum's vector component, and the spin quantum number is the intrinsic angular momentum. Each orbital is described by the variables n,l,m, and due to pauli exclusion principle, no electron in the system can share the same value for all four variables, hence each orbital can take up to two electrons.

The wave function is usually expressed using polar coordinates, due to the fact that it is possible to seperate the wave function into three functions, each taking in only one value of the coordinate. The radial component is a function with exponential decay, and the angular component are formed from spherical harmonics, a set of functions in which combinations of these functions generates arbitary functions defined on a sphere, like fourier series. Spherical harmonics plays a role in the construction of the wave functions due to its link to producing standing waves on the surface of a sphere, which is applicable in this situation as well.

Notice that the wavefunction can only be explicitly expressed for single electron atoms (Hydrogen-like). If there are multiple electrons, the repulsion between the electrons would cause the wave functions of the electrons to be entangled, resulting in our inability to write an explicit wave functions for it. We can, however, attempt to approximate it, using various methods such as the hartree-fock method.

In terms of programming, it opened me to the world of symbolic computation. In conventional programming, the closest thing resembling symbols are variables, which must hold a value. Symbols, on the other hand, is just a placeholder, allowing different possible methods of calculation (such as evaluating equality). Another feature that this code uses is numpy's capability of vectorization. It is essentially batch processing an array of data with a single function call, which is many times faster tha calling the function iteratively for every single data.

## Note #1

Although the 2D graphs below might allude to the idea that the electron is mostly found in the near the nucleus, due to the bright colour, this is in fact no the case. This is because although the outer regions of the atom is much dimmer, they cover a greater space. The region near the nucleus has higher probability density, but smaller space, so it is actually quite unlikely to find the electron in those region.

To illustrate my point, here is the probability of find a electron at certain distance from the nucleus for the first three s-orbitals, extracted from "General Chemistry: Principles, Patterns, and Applications - 6.5 Atomic Orbitals and Their Energies":
![alt text](Reference/s_orbitals.jpg)

## Note #2

A hydrogen wave function also comes in two flavours, the complex atomic orbital, and a real atomic orbital. When describing the atomic orbitals of an electron with its four values (n,l,m,s), one would obtain complex atomic orbitals, which is shown below, extracted from the wikipeida page on magnetic quantum number.
![alt text](Reference/AO.png)
As you can see, many of their shapes are the same, except their phase difference (look at how the colour evolves going round the loop). To make better visualization, complex atomic orbitals are combined to form superpositions, whose wave function lies in the real domain. This is the real orbital, which will be used in the following code. 

This is in fact equilavent to the complex type, and the reason it is so is quite complicated. It is alluded by the fact that electrons can be in superposition between orbitals, and using the complex atomic orbital or real atomic orbitalis simply the matter of picking which [basis](https://en.wikipedia.org/wiki/Basis_(linear_algebra)) to use.

### Either way, this is very important because it sheds light on what the orbitals we are familiar with actually are. They are useful in chemistry for different electron models, but they are just a possible selection of basis which makes visualization easier. The complex atomic orbital, though looking quite different, will still yield correct answers when used in calculations.

## Please run every block of code below

In [ ]:
# The code's dependencies are declared here
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
from sympy.physics.hydrogen import Psi_nlm, R_nl #might be unnecessary
from sympy.functions.special.spherical_harmonics import Znm
from sympy import Symbol
from sympy.utilities.lambdify import lambdify
from mpmath import atan2

import ipyvolume as ipv

## Constructing the wave function

In this part, the real wavefunction is constructed. Adjust the atomic orbital being plotted by typing in the principal quantum number (n), the azimuthal quantum number (l) and the magnetic quantum number (m) of the orbital. This would generate the wave function of the orbital in spherical coordinate, using the library sympy.

In [ ]:
r = Symbol("r", real=True, positive=True) #radius
phi = Symbol("phi", real=True) #azimuthal angle
theta = Symbol("theta", real=True) #polar angle
Z = Symbol("Z", positive=True, integer=True, nonzero=True) #atomic number

#Variables - n,1,m
n,l,m = 5,2,2

#Construction of complex wave function, hard to visualize
#psi = Psi_nlm(n,l,m,r,phi,theta,Z)

#Construction of real wave function, used in the following code
psi = R_nl(n, l, r, Z)*Znm(l,m,theta,phi).expand(func=True)
print("Psi: ", psi)

## Plotting the prabability density in 2D

A vertical (parallel to z-axis) 2D slice of the plot of probability density is rendered via the code below, which produces a graph where a light colour means a higher probability density in that region of space.

This is handelled using numpy vectorization, which utilizes a huge tensor (multi-dimensional array) to store the x and y coordinate of every point of the system we wish to record the probability density. The cartesian coordinate is then transformed to the polar coordinate. And the tensor of coordinates are substituted into the wave function to obtain the tensor of prabability amplitude, and then the tensor of probability density, which is directly fed into the plotting library to produce the plot.

Utilizing this approach over the native python for-loop, going through each pixel, enables magnitudes of speed up, which allows a detailed plot to be rendered in reasonable time.

The variables that can be adjusted are maxi, the maximum distance to render from nucleus, and resolution, the number of points to measure in a dimension.

In [ ]:
#NUMPY VECTORIZATION, FAST
#Variables to adjust
maxi = 40
resolution = 180

#Setting up equally spaced points around the nucleus
base = np.linspace(-maxi, maxi, resolution)[:,np.newaxis]
x2 = np.tile(base, (1,resolution))
y2 = np.swapaxes(x2,0,1)
total = np.concatenate((x2[np.newaxis,:],y2[np.newaxis,:]), axis=0)

#Converting cartesian coordinates into polar coordinates
r2 = np.linalg.norm(total, axis=0)
theta2 = np.arctan2(total[1],total[0])

#Substitution of values
psi2 = psi.subs({Z: 1, phi: 0})
lam_psi = lambdify([r,theta], psi2, 'numpy')
image1 = lam_psi(r2,theta2)
#Convertion of prabability amplitude to probability density
image1 = np.real(image1*np.conj(image1))

#Plotting
plt.imshow(image1, cmap="gist_gray")
plt.colorbar()

In [ ]:
#NATIVE APPROACH, VERY SLOW
maxi = 40
resolution = 20
image = np.empty([resolution,resolution])

for x in range(resolution):
    for y in range(resolution):
        x1 = 2*maxi*x/resolution-maxi
        y1 = 2*maxi*y/resolution-maxi
        res = psi.subs({Z: 1, r: (x1**2+y1**2)**0.5, phi: 0, theta: atan2(y1,x1)})
        image[x,y] = res * res.conjugate()
        
plt.imshow(image, cmap="gist_gray")
plt.colorbar()

## Plotting the prabability density in 3D

Two modes are availiable to visualize the probability density in 3D. The first method is to plot the regions of space with different colour and opacities based on how high the probability density is. The second method is to draw a 3D contour around the regions of high probability density, creating what is called an isosurface. The volume bounded by the isosurface is where the electron is mainly "located" at (they don't locate at a single point).

This code is similarly written using numpy vectorization, in order to speed up the execution.

The first part of the code sets up the tensor which stores the probability density at points around the nucleus. Run the second part of the code to plot the entire probability density with colour and opacities. To draw out the isosurface, run the third part of the code. Adjust the slide's position beneath the plot to adjust the threshold for a point to reside within the isosurface.

In [ ]:
#Variables to adjust
maxi = 60
resolution = 160

base = np.linspace(-maxi, maxi, resolution)[:,np.newaxis,np.newaxis]
x2 = np.tile(base, (1,resolution,resolution))
y2 = np.swapaxes(x2,0,1)
z2 = np.swapaxes(x2,0,2)

total = np.concatenate((x2[np.newaxis,:],y2[np.newaxis,:],z2[np.newaxis,:]), axis=0)

r2 = np.linalg.norm(total, axis=0)
#Alternative theta calculation
#theta3 = np.abs(np.arctan2(np.linalg.norm(total[:2], axis=0),-total[2]))
np.seterr(all='ignore')
theta2 = np.arctan(np.divide(total[2],np.linalg.norm(total[:2], axis=0))) + np.pi/2
phi2 = np.arctan2(total[1],total[0])

psi2 = psi.subs({Z: 1})
lam_psi = lambdify([r,phi,theta], psi2, 'numpy')
image2 = np.square(lam_psi(r2,phi2,theta2)).real
#Scaling the probability density up
image2 *=1000000000

In [ ]:
#Volume with opacities
ipv.figure()
ipv.volshow(image2)
ipv.show()

In [ ]:
#Isosurface
ipv.figure()
ipv.plot_isosurface(image2)
ipv.show()